In [1]:
from matplotlib import pyplot as plt
import numpy as np
import sklearn
import cv2
import math
from random import randint

In [2]:
global img
img = cv2.imread("SAFlag.jpg")
global img_copy
img_copy = img.copy()
global k_total
global h
global w
global c
h, w, c = img.shape

In [3]:
def trackbar(x):
    global k 
    k = cv2.getTrackbarPos("K_Val","image")

In [4]:
#randomly select k point
def K_generator(k_total):
    global k_point
    k_point = []
    for k in range(k_total):   
        x = randint(0,h - 1)
        y = randint(0,w - 1)
        k_point.append(img_copy[x,y]) 
    

In [5]:
def voronoi_region(event, x, y, flag, param):
    #k_point = K_generator(k_total)
    h, w, c = img.shape
    cv2.imshow('image',img_copy)
    idx = 0
    ssd = 0
    flag = 0
       
    if event == cv2.EVENT_LBUTTONUP:     
        while(not flag):
            threshold = round(255/k_total)
            k_sum = np.zeros([3,k_total])
            save = np.zeros(k_total,dtype=int)
            means = np.zeros([3,k_total])
            
            k_cluster = {idx:[] for idx in range(k_total)} #generate 5 empty list as cluster lists
            print("please wait for 30 ~ 50 seconds")
            print(k_cluster)
            for x in range(h):
                for y in range(w):
                    current_color = img_copy[x][y]
                    min_diff = 442
                    mini = -1
                    for idx in range(k_total):
                        k_color = k_point[idx] #get current k color
                        b = np.square(abs(int(current_color[0]) - int(k_color[0])))
                        g = np.square(abs(int(current_color[1]) - int(k_color[1])))
                        r = np.square(abs(int(current_color[2]) - int(k_color[2])))
                        ssd = math.sqrt(b + g + r)
                #get color difference
                        if (ssd < min_diff):
                                min_diff = ssd
                                mini = idx               #print(index)
                    for idx in range(3):
                        k_sum[idx][mini] = k_sum[idx][mini] + current_color[idx]
                    save[mini] = save[mini] + 1
                    
                    k_cluster[mini].append([x,y]) #get the corresponding k color of the pixel
        
                
            for idx in range(3):
                means[idx] = [i/j for i, j in zip(k_sum[idx],save)]
        
            for idx in range(k_total):
                b_diff = abs(means[0][idx]-k_point[idx][0])
                g_diff = abs(means[1][idx]-k_point[idx][1])
                r_diff = abs(means[2][idx]-k_point[idx][2])
                
                
                if (b_diff > threshold or g_diff > threshold or r_diff > threshold):                  
                    for idx in range(k_total):
                        k_point[idx] = [round(means[0][idx]),round(means[1][idx]),round(means[2][idx])]
                    flag = 0
                    break
                else:
                    flag = 1
                    for idx in range(k_total):
                        for c_col in k_cluster[idx]:
                            img_copy[c_col[0]][c_col[1]] = [round(means[0][idx]),round(means[1][idx]),round(means[2][idx])]
               


In [6]:
k_total = randint(2,20)

cv2.namedWindow('image')
cv2.createTrackbar('K_Val','image',2,20,K_generator)
cv2.setMouseCallback('image',voronoi_region)

K_generator(k_total) #get k

#cv2.imshow('img',img_copy)
k = cv2.waitKey(0) & 0XFF
if k == 27:
    cv2.destroyAllWindows()

please wait for 30 ~ 50 seconds
the total number of K is listed below: (it is a random number between 2 to 20)
{0: [], 1: [], 2: [], 3: []}
